# Data Processing and Export

This notebook fetches and processes data from a specified database and exports it to a CSV file. The data includes item information and sales prices, which are then filtered and merged based on specific conditions. Finally, the processed data is exported to a CSV file with a unique filename based on the current timestamp. The process is scheduled to run daily at 15:02.


## Import Libraries

First, we import the necessary libraries for data processing and database connection.


In [ ]:
import schedule
import time
import pandas as pd
import datetime
import pyodbc
import matplotlib.pyplot as plt
import seaborn as sns

## Database Connection
We establish a connection to the NCT2-RBO database using the specified server, database name, username, and password.

In [ ]:
server = 'your_server_name' 
database = 'your_database_name' 
username = 'your_username' 
password = 'your_password'  

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password)
cursor = cnxn.cursor()


## Fetch Data
We execute SQL queries to fetch data from the Item and SalesPrice tables.

In [ ]:
query_item = "SELECT * FROM [Item];"
df_item = pd.read_sql(query_item, cnxn)

query_sales = "SELECT * FROM [SalesPrice];"
df_sales = pd.read_sql(query_sales, cnxn)


## Data Preprocessing
We convert the Starting Date and Ending Date columns to datetime format, filter the DataFrame based on date range, and merge the filtered sales data with item data.

In [ ]:
    # Convert 'Starting Date' and 'Ending Date' to datetime format
df_sales['Starting Date'] = pd.to_datetime(df_sales['Starting Date'], errors='coerce').dt.date
df_sales['Ending Date'] = pd.to_datetime(df_sales['Ending Date'], errors='coerce')

    # Define date range for filtering
date_from = pd.to_datetime('2022-01-01').date()
date_to = pd.to_datetime('1753-01-01 00:00:00.000')

    # Filter the DataFrame based on date range
filtered_df = df_sales[(df_sales['Starting Date'] < date_from) & (df_sales['Ending Date'] == date_to)]

    # Select specific columns from the filtered DataFrame
filtered_df = filtered_df[['ItemNo', 'Starting Date', 'Ending Date', 'SalesCode']]

    # Select specific columns from the item DataFrame
state = df_item[['No', 'ProductStatus', 'DivisionCode']]

# Merge the two DataFrames based on 'ItemNo' and 'No'
merged_df_with_status = pd.merge(filtered_df, state, left_on='ItemNo', right_on='No', how='left')

 # Select specific columns from the merged DataFrame
merged_df_with_status_filtered = merged_df_with_status[['ItemNo', 'SalesCode', 'DivisionCode', 'Starting Date', 'Ending Date', 'ProductStatus']]


## Replace Codes with Descriptions
We replace numerical product status codes and division codes with their corresponding descriptions

In [ ]:
    # Replace numerical product status codes with their corresponding status descriptions
replace_values = {
        0: 'NO status', 1: 'New', 2: 'Live', 3: 'Delete', 4: 'Clean',
        5: 'Block', 6: 'Block Consumer', 7: 'Block BOM', 8: 'Transfer & Purchase',
        9: 'All-Exp-SA', 10: 'Purchase Block', 11: 'Multi-Location'
    }
merged_df_with_status_filtered = merged_df_with_status_filtered.replace({"ProductStatus": replace_values})

    # Replace numerical division codes with their corresponding division descriptions
replace_values_Division = {
        1: 'FMCG', 2: 'Fresh Food', 3: 'General Merchandising', 4: 'Tenants',
        5: 'Overhead', 12: 'Penalty', 99: 'ETax'
    }
merged_df_with_status_filtered = merged_df_with_status_filtered.replace({"DivisionCode": replace_values_Division})



## Visualization : Distribution of Product Statuses
Generate a bar plot showing the distribution of product statuses.

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=merged_df_with_status_filtered, x='ProductStatus', order=merged_df_with_status_filtered['ProductStatus'].value_counts().index)
plt.title('Distribution of Product Statuses')
plt.xlabel('Product Status')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


### Visualization : Distribution of Starting Dates
This block generates a histogram showing the distribution of `Starting Date` values in the filtered dataset. It helps in understanding the frequency of item start dates before the specified date.


In [ ]:
    # Visualization 1: Distribution of Starting Dates
plt.figure(figsize=(10, 6))
plt.hist(filtered_df['Starting Date'], bins=30, edgecolor='k', alpha=0.7)
plt.title('Distribution of Starting Dates')
plt.xlabel('Starting Date')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


## Visualization : Heatmap of Product Status by Division
Generate a heatmap showing the relationship between product status and division.

In [ ]:
plt.figure(figsize=(12, 8))
status_by_division = merged_df_with_status_filtered.pivot_table(index='DivisionCode', columns='ProductStatus', aggfunc='size', fill_value=0)
sns.heatmap(status_by_division, annot=True, fmt='d', cmap='viridis')
plt.title('Heatmap of Product Status by Division')
plt.xlabel('Product Status')
plt.ylabel('Division Code')
plt.xticks(rotation=45)
plt.show()


## Export Data to CSV
We export the final processed DataFrame to a CSV file with a unique filename based on the current timestamp.

In [ ]:
now = datetime.datetime.now()
now_str = now.strftime("%Y-%m-%d-%H-%M-%S")
filename = f'./NO_EndDate_status_{now_str}.csv'
merged_df_with_status_filtered.to_csv(filename, index=False)


## Schedule the Task
We schedule the process_data function to run daily at 15:02 and keep the script running to execute the scheduled task.

In [ ]:
schedule.every().day.at("15:02").do(process_data)

while True:
    schedule.run_pending()
    time.sleep(1)
